# Search

Compare different methods of searching for a substring in another string.

In [1]:
%load_ext autoreload
%autoreload 2

## Import data

In [2]:
import pandas as pd

In [3]:
DATA_FILE = '../data/southpark-all-seasons.csv'

df = pd.read_csv(DATA_FILE)
lowercase = lambda x: str(x).lower()
df.rename(lowercase, axis='columns', inplace=True)

In [4]:
df.head()

,season,episode,character,line
0,10,1,Stan,"You guys, you guys! Chef is going away. \n"
1,10,1,Kyle,Going away? For how long?\n
2,10,1,Stan,Forever.\n
3,10,1,Chef,I'm sorry boys.\n
4,10,1,Stan,"Chef said he's been bored, so he joining a gro..."


## Search parameters

In [5]:
string_to_find = "They killed Kenny"
test_a = "They didn't kill Kenny"
test_b = "You guys, you guys! Chef is going away to see kenny."
test_c = "they killed him, Kenny"
test_d = "They did it! They killed him!"

tests = [test_a, test_b, test_c, test_d]

## 1st method: brute-force

In [6]:
def find_brute_force(string, string_to_find):
    return all(keyword.lower() in string.lower() for keyword in string_to_find.lower().split())

### Test strings

In [7]:
%%time
for test_str in tests:
    print(test_str, find_brute_force(test_str, string_to_find))

They didn't kill Kenny False
You guys, you guys! Chef is going away to see kenny. False
they killed him, Kenny True
They did it! They killed him! False
CPU times: user 740 µs, sys: 639 µs, total: 1.38 ms
Wall time: 865 µs


In [8]:
%%timeit
for test_str in tests:
    find_brute_force(test_str, string_to_find)

4.95 µs ± 80.4 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


### Data strings

In [9]:
%%time
for test_str in df.line:
    res = find_brute_force(test_str, string_to_find)
    if res:
        print(test_str.replace('\n', ''))

Aaaah!  Oh my God, they killed Kenny!
Oh my God. Oh my God, they- they killed Kenny!
Oh my God, they killed Kenny! YOU BASTARDS!
No, not they, YOU! Look what your ziplining idea has done! You killed Kenny, YOU're the bastard!
Oh my God! They've killed Kenny!
Cartman, they killed Kenny!
Huh, Oh my God, they killed Kenny! You bastard! 
Oh my God, they killed Kenny!
Wha-? Oh my God, they killed Kenny! You Bastards!
Oh my God, they killed Kenny! You bastards!
Oh my God, they killed Kenny. You Bastard!!!
Oh my God! They killed Kenny!
Oh my god, they've killed Kenny!
Oh my God, they killed Kenny.
Oh my God, they killed Kenny!
Oh my God, they've killed Kenny.
Oh my God, they've killed Kenny!
Oh my God, they've killed Kenny!
Oh my God, they killed Kenny!
Oh, my God! They've killed Kenny!
Oh my God! They've killed Kenny!
Oh, my God! They killed Kenny!
Oh my God, they've killed Kenny!
Oh my God, they killed Kenny!
Oh my God, they killed Kenny!
Oh my God, they killed Kenny.
Oh my God, they killed

In [10]:
%%timeit
for test_str in df.line:
    find_brute_force(test_str, string_to_find)

75.8 ms ± 554 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


## Regex

In [11]:
import re

In [12]:
def find_regex(string, string_to_find):
    return set(re.findall(r'|'.join(string_to_find.lower().split()), string.lower(), re.IGNORECASE))

### Test strings

In [13]:
%%time

truth_len = len(string_to_find.split())

for test_str in tests:
    print(test_str, truth_len == len(find_regex(test_str, string_to_find)))

They didn't kill Kenny False
You guys, you guys! Chef is going away to see kenny. False
they killed him, Kenny True
They did it! They killed him! False
CPU times: user 1.25 ms, sys: 1 ms, total: 2.25 ms
Wall time: 1.51 ms


In [14]:
%%timeit
for test_str in tests:
    find_regex(test_str, string_to_find)

13.3 µs ± 179 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


### Data strings

In [15]:
%%time
for test_str in df.line:
    res = (truth_len == len(find_regex(test_str, string_to_find)))

    if res:
        print(test_str.replace('\n', ''))

Aaaah!  Oh my God, they killed Kenny!
Oh my God. Oh my God, they- they killed Kenny!
Oh my God, they killed Kenny! YOU BASTARDS!
No, not they, YOU! Look what your ziplining idea has done! You killed Kenny, YOU're the bastard!
Oh my God! They've killed Kenny!
Cartman, they killed Kenny!
Huh, Oh my God, they killed Kenny! You bastard! 
Oh my God, they killed Kenny!
Wha-? Oh my God, they killed Kenny! You Bastards!
Oh my God, they killed Kenny! You bastards!
Oh my God, they killed Kenny. You Bastard!!!
Oh my God! They killed Kenny!
Oh my god, they've killed Kenny!
Oh my God, they killed Kenny.
Oh my God, they killed Kenny!
Oh my God, they've killed Kenny.
Oh my God, they've killed Kenny!
Oh my God, they've killed Kenny!
Oh my God, they killed Kenny!
Oh, my God! They've killed Kenny!
Oh my God! They've killed Kenny!
Oh, my God! They killed Kenny!
Oh my God, they've killed Kenny!
Oh my God, they killed Kenny!
Oh my God, they killed Kenny!
Oh my God, they killed Kenny.
Oh my God, they killed

In [16]:
%%timeit
for test_str in df.line:
    find_regex(test_str, string_to_find)

316 ms ± 7.91 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Aho-Corasick algorithm

This algorithm is case-sensitive.

In [17]:
from aho_corasick import aho_corasick

### Test strings

In [18]:
%%time

truth_len = len(string_to_find.split())

for test_str in tests:
#     print(aho_corasick(test_str.lower(), string_to_find.lower().split()))
    print(test_str, truth_len == len(set(aho_corasick(test_str.lower(), string_to_find.lower().split()))))

They didn't kill Kenny False
You guys, you guys! Chef is going away to see kenny. False
they killed him, Kenny True
They did it! They killed him! False
CPU times: user 430 µs, sys: 109 µs, total: 539 µs
Wall time: 456 µs


In [19]:
%%timeit

for test_str in tests:
    truth_len == len(set(aho_corasick(test_str.lower(), string_to_find.lower().split())))

144 µs ± 2.17 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


### Data strings

In [20]:
%%time
for test_str in df.line:
    res = (truth_len == len(set(aho_corasick(test_str.lower(), string_to_find.lower().split()))))
    if res:
        print(test_str.replace('\n', ''))

Aaaah!  Oh my God, they killed Kenny!
Oh my God. Oh my God, they- they killed Kenny!
Oh my God, they killed Kenny! YOU BASTARDS!
No, not they, YOU! Look what your ziplining idea has done! You killed Kenny, YOU're the bastard!
Oh my God! They've killed Kenny!
Cartman, they killed Kenny!
Huh, Oh my God, they killed Kenny! You bastard! 
Oh my God, they killed Kenny!
Wha-? Oh my God, they killed Kenny! You Bastards!
Oh my God, they killed Kenny! You bastards!
Oh my God, they killed Kenny. You Bastard!!!
Oh my God! They killed Kenny!
Oh my god, they've killed Kenny!
Oh my God, they killed Kenny.
Oh my God, they killed Kenny!
Oh my God, they've killed Kenny.
Oh my God, they've killed Kenny!
Oh my God, they've killed Kenny!
Oh my God, they killed Kenny!
Oh, my God! They've killed Kenny!
Oh my God! They've killed Kenny!
Oh, my God! They killed Kenny!
Oh my God, they've killed Kenny!
Oh my God, they killed Kenny!
Oh my God, they killed Kenny!
Oh my God, they killed Kenny.
Oh my God, they killed

In [21]:
%%timeit
for test_str in df.line:
    aho_corasick(test_str.lower(), string_to_find.lower().split())

3.29 s ± 63 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
